In [0]:
import requests
import uuid
import json
from azure.storage.blob import BlobServiceClient
import torch
import sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration


# Test Transformers
#tokenizer = transformers.BertTokenizer.from_pretrained('google/bert-base-uncased')
#print("Transformers test tokenizer:\n", tokenizer)

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

print("Model ready")

#storage_account_name = "zihdssnbxpocblob0"
#blob_container_name = "translation"
#input_document_name = "DE-EN-alligned.txt"
#output_document_name = "DE-EN-pred-groundtruth.txt"

# File paths and Blob Storage URLs
input_blob_url = "https://zihdssnbxpocblob0.blob.core.windows.net/nlp-tests/DE-EN-alligned.txt"
output_blob_url = "https://zihdssnbxpocblob0.blob.core.windows.net/nlp-tests-outputs/DE-EN-pred-groundtruth.txt"



# Create a BlobServiceClient to interact with the Blob Storage
blob_service_client = BlobServiceClient.from_connection_string("DefaultEndpointsProtocol=https;AccountName=zihdssnbxpocblob0;AccountKey=57NNKs6efrO4ub4yxjjXkRsVvXDbKlOfhOdnufMccGzNf7i2QGGUNhvDANo+Ma6rlCyBB5zyFebY+AStHe09nw==;EndpointSuffix=core.windows.net")

# Download input file from Azure Blob Storage
input_container = input_blob_url.split('/')[3]
input_blob_name = input_blob_url.split('/')[-1]
input_container_client = blob_service_client.get_container_client(input_container)
input_blob_client = input_container_client.get_blob_client(input_blob_name)

# Translate and write to output file
translated_lines = []

#with input_blob_client.download_blob() as blob:
#content = blob.readall().decode('utf-8')
#lines = content.split('\n')
response = input_blob_client.download_blob()
content = response.readall().decode('utf-8')
lines = content.split('\n')
i=1
for line in lines:
    string_1, string_2 = line.strip().split('\t')

    # FLAN T5 TRANSLATION

    input_text = "translate German to English: " + string_1
    
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

    outputs = model.generate(input_ids)
    translated_string =tokenizer.decode(outputs[0])

    # translated string is the string_1 translation
        
    translated_lines.append(translated_string + '\t' + string_2)

    print("Translation number", i, "\n", translated_string + ' -- ' + string_2)
    i += 1

# Upload the translated output to Azure Blob Storage
output_container = output_blob_url.split('/')[3]
output_blob_name = output_blob_url.split('/')[-1]
output_container_client = blob_service_client.get_container_client(output_container)
output_blob_client = output_container_client.get_blob_client(output_blob_name)
output_blob_client.upload_blob('\n'.join(translated_lines))

print("Translation completed. Translated text saved to", output_blob_url)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-2232309253259152> in <module> 
 12 #print("Transformers test tokenizer:\n", tokenizer) 
 13 
 ---> 14 tokenizer = T5Tokenizer . from_pretrained ( "t5-base" ) 
 15 model = T5ForConditionalGeneration . from_pretrained ( "google/flan-t5-large" , device_map = "auto" ) 
 16 

 /databricks/python/lib/python3.8/site-packages/transformers/tokenization_utils_base.py in from_pretrained (cls, pretrained_model_name_or_path, *init_inputs, **kwargs) 
 1653 # Try to get the tokenizer config to see if there are versioned tokenizer files. 
 1654 fast_tokenizer_file = FULL_TOKENIZER_FILE
 -> 1655 resolved_config_file = get_file_from_repo(
 1656 pretrained_model_name_or_path , 
 1657 TOKENIZER_CONFIG_FILE , 

 /databricks/python/lib/python3.8/site-packages/transformers/file_utils.py in get_file_from_repo (path_or_repo, filename, cache_dir, force_download, resume_download, proxies, use_auth_token, revision, local_files_only) 
 2233 try : 
 2234 # Load from URL or cache if already cached 
 -> 2235 resolved_file = cached_path(
 2236 resolved_file , 
 2237 cache_dir = cache_dir , 

 /databricks/python/lib/python3.8/site-packages/transformers/file_utils.py in cached_path (url_or_filename, cache_dir, force_download, proxies, resume_download, user_agent, extract_compressed_file, force_extract, use_auth_token, local_files_only) 
 1844 if is_remote_url ( url_or_filename ) : 
 1845 # URL, so get it from the cache (downloading if necessary) 
 -> 1846 output_path = get_from_cache(
 1847 url_or_filename , 
 1848 cache_dir = cache_dir , 

 /databricks/python/lib/python3.8/site-packages/transformers/file_utils.py in get_from_cache (url, cache_dir, force_download, proxies, etag_timeout, resume_download, user_agent, use_auth_token, local_files_only) 
 2100 )
 2101 else : 
 -> 2102 raise ValueError(
 2103 "Connection error, and we cannot find the requested files in the cached path." 
 2104 " Please try again or make sure your Internet connection is on." 

 ValueError : Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [0]:
import torch
import transformers

print("PyTorch version:", torch.__version__)
print("Transformers version:", transformers.__version__)

PyTorch version: 1.10.2+cpu
Transformers version: 4.16.2